<a href="https://colab.research.google.com/github/toanpt74/COLAB_RD/blob/main/VAE_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Input, Conv2D, Conv2DTranspose, Flatten, Dense, Reshape, ReLU, Lambda, BatchNormalization
import os
import keras
from keras import layers
from tensorflow.python.keras.callbacks import ModelCheckpoint

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def preprocess(array):
    """
    Normalizes the supplied array and reshapes it into the appropriate format.
    """
    array = array.astype("float32") / 255.0
    array = np.reshape(array, (len(array), 28, 28, 1))
    return array

def noise(array):
    """
    Adds random noise to each image in the supplied array.
    """
    noise_factor = 0.4
    noisy_array = array + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size=array.shape
    )
    return np.clip(noisy_array, 0.0, 1.0)

def display(array1, array2):
    """
    Displays ten random images from each one of the supplied arrays.
    """
    n = 10
    indices = np.random.randint(len(array1), size=n)
    images1 = array1[indices, :]
    images2 = array2[indices, :]
    plt.figure(figsize=(20, 4))
    for i, (image1, image2) in enumerate(zip(images1, images2)):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(image1.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(image2.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

(train_data, _), (test_data, _) = mnist.load_data()

# Normalize and reshape the data
train_data = preprocess(train_data)
test_data = preprocess(test_data)
# Create a copy of the data with added noise
noisy_train_data = noise(train_data)
noisy_test_data = noise(test_data)
# Display the train data and a version of it with added noise
#display(train_data, noisy_train_data)

ROW = 28
COL = 28
def pre_image(image_filename):
    print("Load file:" + image_filename)
    img_raw = tf.io.read_file(image_filename)
    image = tf.image.decode_bmp(img_raw)
    image = tf.cast(image, dtype=tf.float32)
    image = tf.image.resize(image, (ROW, COL))
    image = image / 255.0
    image = tf.reshape(image, shape=(ROW, COL, 1,)) #shape(hang, cot)
    print(image.shape)
    return image

AUTO = tf.data.experimental.AUTOTUNE
original_dim = 28*28
input_shape = (28,28,1)
intermediate_dim = 128
batch_size = 128
latent_dim = 32
epochs = 200
def display(array1, array2):
    """
    Displays ten random images from each one of the supplied arrays.
    """
    n = 10
    indices = np.random.randint(len(array1), size=n)
    images1 = array1[indices, :]
    images2 = array2[indices, :]
    plt.figure(figsize=(20, 4))
    for i, (image1, image2) in enumerate(zip(images1, images2)):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(image1.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(image2.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()
class Sampling(layers.Layer):
    def call(self, inputs):
        mu, sigma = inputs
        batch = tf.shape(mu)[0]
        dim = tf.shape(mu)[1]
        epsilon = keras.backend.random_normal(shape=(batch, dim))
        z = mu + tf.exp(0.5 * sigma) * epsilon
        return z


def encoder_layer(inputs, latent_dim):
    # Block-1
    x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu', name='conv_1')(inputs)
    x = BatchNormalization(name='bn_1')(x)

    # Block-2
    x = Conv2D(64, kernel_size=3, strides=2, padding='same', activation='relu', name='conv_2')(x)
    x = BatchNormalization(name='bn_2')(x)
    # Block-3
    x = Conv2D(64, kernel_size=3, strides=2, padding='same', activation='relu', name='conv_3')(x)
    x = BatchNormalization(name='bn_3')(x)

    # Block-4
    x = Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu', name='conv_4')(x)
    x = BatchNormalization(name='bn_4')(x)
    batch_3 = x #BatchNormalization()(x)
    #Final Block
    x = Flatten()(batch_3)
    mu = layers.Dense(latent_dim, name='latent_mu')(x)
    sigma = layers.Dense(latent_dim, name='latent_sigma')(x)
    return mu, sigma, batch_3.shape

def encoder_model(latent_dim, input_shape):
    inputs = Input(shape=input_shape)
    mu, sigma, conv_shape = encoder_layer(inputs, latent_dim=latent_dim)
    z = Sampling()((mu, sigma))
    model = Model(inputs, outputs=[mu, sigma, z], name='Encoder')
    model.summary()
    return model, conv_shape

def decoder_layer(inputs, conv_shape):
    units = conv_shape[1] * conv_shape[2] * conv_shape[3]
    x = Dense(units, name='dense_1')(inputs)
    x = BatchNormalization()(x)
    x = Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(x)
    # Block-1
    x = Conv2DTranspose(filters=64, kernel_size=3, strides=1, padding='same', activation='relu',name='conv_transpose_1')(x)
    x = BatchNormalization(name='bn_1')(x)
    # Block-2
    x = Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='relu',name='conv_transpose_2')(x)
    x = BatchNormalization(name='bn_2')(x)

    # Block-3
    x = Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='relu',
                               name='conv_transpose_3')(x)
    x = BatchNormalization(name='bn_3')(x)
    # Block-4
    x = Conv2DTranspose(filters=32, kernel_size=3, strides=1, padding='same', activation='relu',
                               name='conv_transpose_4')(x)
    x = BatchNormalization(name='bn_4')(x)
    outputs = Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding='same', activation='sigmoid',name='out')(x)
    return outputs

def decoder_model(latent_dim, conv_shape):
    inputs = Input(shape=(latent_dim,))
    outputs = decoder_layer(inputs, conv_shape)
    model = Model(inputs, outputs, name='Decoder')
    model.summary()
    return model

def kl_reconstruction_loss(inputs, outputs, mu, sigma):
    kl_loss = 1 + sigma - tf.square(mu) - tf.math.exp(sigma)
    return tf.reduce_mean(kl_loss) * -0.5


def vae_model(encoder, decoder, input_shape):
    inputs = keras.layers.Input(shape=input_shape)
    mu, sigma, z = encoder(inputs)
    reconstructed = decoder(z)
    model = keras.Model(inputs=inputs, outputs=reconstructed)
    loss = kl_reconstruction_loss(inputs, z, mu, sigma)
    model.add_loss(loss)
    return model


def get_models(input_shape, latent_dim):
    encoder, conv_shape = encoder_model(latent_dim=latent_dim, input_shape=input_shape)
    decoder = decoder_model(latent_dim=latent_dim, conv_shape=conv_shape)
    vae = vae_model(encoder, decoder, input_shape=input_shape)
    return encoder, decoder, vae

encoder, decoder, vae = get_models(input_shape=input_shape, latent_dim=latent_dim)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
loss_metric = keras.metrics.Mean()
mse_loss = keras.losses.MeanSquaredError()

x_train =  tf.data.Dataset.from_tensor_slices((train_data))

train_ds = (x_train
            .shuffle(len(x_train))
                # .map(pre_image, num_parallel_calls=AUTO)
                .batch(batch_size)
                .prefetch(buffer_size=AUTO))

model_path =r'E:\PROJECT-AI\ImageClassification\models'

# for epoch in range(epochs):
#     print('Start of epoch %d' % (epoch,))
#     for step, x_batch_train in enumerate(train_ds):
#         with tf.GradientTape() as tape:
#             reconstructed = vae(x_batch_train)
#             flattened_inputs = tf.reshape(x_batch_train, shape=[-1])
#             flattened_outputs = tf.reshape(reconstructed, shape=[-1])
#             loss = mse_loss(flattened_inputs, flattened_outputs) * COL * ROW
#             loss += sum(vae.losses)
#
#         grads = tape.gradient(loss, vae.trainable_weights)
#         optimizer.apply_gradients(zip(grads, vae.trainable_weights))
#         loss_metric(loss)
#         print('Epoch: %s step: %s mean loss = %s' % (epoch, step, loss_metric.result().numpy()))
#     if epoch % 5 == 0:
#         vae.save(f'VAE_MNIST\\model_{epoch}__{loss}', save_format="tf")

model = tf.keras.models.load_model(r'E:\ToanPT\1.Code_train_Unet\VAE_MNIST\model_195__4.01559591293335')

re = model.predict(x_train)

display(x_train,re)









